# 🎤 Voice-Based Assistant for Email & Meeting Scheduling using OpenAI Function Calling

This notebook demonstrates a voice-controlled assistant that can perform tasks like sending emails and scheduling meetings using OpenAI's function calling feature.

**🔍 What this notebook does:**
- Records your voice command using a microphone.

- Transcribes the audio to text using Whisper (an automatic speech recognition model by OpenAI).

- Processes the command using OpenAI GPT-4o, which decides whether to send an email or schedule a meeting.

- Executes the relevant function and displays the response.

- Provides a user-friendly interface via Gradio.

**💡 Example Commands:**
- "Send an email to John with the subject Project Update and the body Please check the latest report."

- "Schedule a meeting with Alice and Bob on April 10 at 3 PM."


**🛠️ 1. Install Required Libraries**

In [17]:
# Install all necessary libraries
!pip install openai whisper torch torchaudio gTTS google-api-python-client google-auth google-auth-oauthlib google-auth-httplib2 gradio scipy --quiet

**🔐 2. Setup API Key**

In [23]:
import openai
from google.colab import userdata

open_ai_key = userdata.get("OPENAI_API_KEY")
# Replace with your own OpenAI key
openai.api_key = open_ai_key

**🧠 3. OpenAI Client and Tool Definitions**

In [25]:
import os
import json
from openai import OpenAI

# Create OpenAI client
def get_client():
    return OpenAI(api_key=openai.api_key)

# Define Email function
def send_email(recipient, subject, body):
    return f"📧 Email sent to {recipient} with subject '{subject}' and body '{body}'."

# Define Meeting Scheduler function
def schedule_meeting(participants, date, time):
    return f"📅 Meeting scheduled with {', '.join(participants)} on {date} at {time}."

# Define tools in OpenAI format
tools = [
    {
        "type": "function",
        "function": {
            "name": "send_email",
            "description": "Send an email",
            "parameters": {
                "type": "object",
                "properties": {
                    "recipient": {"type": "string"},
                    "subject": {"type": "string"},
                    "body": {"type": "string"}
                },
                "required": ["recipient", "subject", "body"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "schedule_meeting",
            "description": "Schedule a meeting",
            "parameters": {
                "type": "object",
                "properties": {
                    "participants": {"type": "array", "items": {"type": "string"}},
                    "date": {"type": "string"},
                    "time": {"type": "string"}
                },
                "required": ["participants", "date", "time"]
            }
        }
    }
]

**🧾 4. Function to Process Text with OpenAI Agent**

In [26]:
def process_query(text):
    client = get_client()
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": text}],
        tools=tools,
        tool_choice="auto"
    )

    message = response.choices[0].message

    if message.tool_calls:
        tool_call = message.tool_calls[0]
        function_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)

        if function_name == "send_email":
            return send_email(**arguments)
        elif function_name == "schedule_meeting":
            return schedule_meeting(**arguments)

    return message.content or "✅ No tool call needed."

**🎙️ 5. Handle Voice Input and Transcription**

In [27]:
import whisper
import scipy.io.wavfile

def voice_assistant(audio_file):
    if audio_file is None:
        return "No audio recorded."

    try:
        sample_rate, data = audio_file
        scipy.io.wavfile.write("recorded.wav", sample_rate, data)

        model = whisper.load_model("base")
        result = model.transcribe("recorded.wav")
        print("🔊 Transcribed Text:", result["text"])

        response = process_query(result["text"])
        return response

    except Exception as e:
        return f"Error: {str(e)}"

**🚀 6. Launch Gradio Interface**

In [29]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("## 🎙️ Voice Assistant for Email & Meeting Scheduling")
    audio_input = gr.Audio(label="Speak your command", type="numpy")
    output_text = gr.Textbox(label="Assistant Response")
    submit_btn = gr.Button("Submit")

    submit_btn.click(fn=voice_assistant, inputs=audio_input, outputs=output_text)

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d1f77947719d6cf111.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
